# United States - Crime Rates - 1960 - 2014

### Introduction:

This time you will create a data 

Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 59 kB/s 
     |████████████████████████████████| 199 kB 78.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=c7e8753f7d454bd7305d62e9bf5dd8581f533363cedae76a72c4c33e98dd8582
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession, functions as f
from pyspark.files import SparkFiles

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/US_Crime_Rates/US_Crime_Rates_1960_2014.csv). 

In [5]:
spark = SparkSession.builder.appName("exercise42").getOrCreate()
url = "https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/US_Crime_Rates/US_Crime_Rates_1960_2014.csv"
spark.sparkContext.addFile(url)

### Step 3. Assign it to a variable called crime.

In [20]:
df = spark.read.csv("file://"+SparkFiles.get("US_Crime_Rates_1960_2014.csv"), header=True, inferSchema=True)
df.show()

+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|Year|Population|   Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|
+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|1960| 179323175| 3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|
|1961| 182992000| 3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|
|1962| 185771000| 3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|
|1963| 188483000| 4109500| 316970| 3792500|  8640|        17650| 116470|            174210| 1086400|      2297800|       408300|
|1964| 191141000| 4564600| 364220| 4200400|  9360|        21420| 130390|            203050| 12132

### Step 4. What is the type of the columns?

In [8]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Violent: integer (nullable = true)
 |-- Property: integer (nullable = true)
 |-- Murder: integer (nullable = true)
 |-- Forcible_Rape: integer (nullable = true)
 |-- Robbery: integer (nullable = true)
 |-- Aggravated_assault: integer (nullable = true)
 |-- Burglary: integer (nullable = true)
 |-- Larceny_Theft: integer (nullable = true)
 |-- Vehicle_Theft: integer (nullable = true)



##### Have you noticed that the type of Year is int64. But pandas has a different type to work with Time Series. Let's see it now.

### Step 5. Convert the type of the column Year to datetime64

In [21]:
df = df.withColumn("Year", f.to_date(f.col("Year").cast("string"), "yyyy"))
df.show()

+----------+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|      Year|Population|   Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|
+----------+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|1960-01-01| 179323175| 3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|
|1961-01-01| 182992000| 3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|
|1962-01-01| 185771000| 3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|
|1963-01-01| 188483000| 4109500| 316970| 3792500|  8640|        17650| 116470|            174210| 1086400|      2297800|       408300|
|1964-01-01| 191141000| 4564600| 364220| 4200400|  9360

### Step 6. Set the Year column as the index of the dataframe

### Step 7. Delete the Total column

In [24]:
df = df.drop(df.Total)
df.show()

+----------+----------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|      Year|Population|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|
+----------+----------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|1960-01-01| 179323175| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|
|1961-01-01| 182992000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|
|1962-01-01| 185771000| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|
|1963-01-01| 188483000| 316970| 3792500|  8640|        17650| 116470|            174210| 1086400|      2297800|       408300|
|1964-01-01| 191141000| 364220| 4200400|  9360|        21420| 130390|            203050| 1213200|      2514400|       

### Step 8. Group the year by decades and sum the values

#### Pay attention to the Population column number, summing this column is a mistake

In [36]:
data_cols = ["Violent", "Property", "Murder", "Forcible_Rape", "Robbery", "Aggravated_assault", "Burglary", "Larceny_Theft", "Vehicle_Theft"]
decade_df =  df.withColumn("decade", f.floor(f.year(df.Year) / 10) * 10).groupby("decade").sum(*data_cols)
decade_df.show()

+------+------------+-------------+-----------+------------------+------------+-----------------------+-------------+------------------+------------------+
|decade|sum(Violent)|sum(Property)|sum(Murder)|sum(Forcible_Rape)|sum(Robbery)|sum(Aggravated_assault)|sum(Burglary)|sum(Larceny_Theft)|sum(Vehicle_Theft)|
+------+------------+-------------+-----------+------------------+------------+-----------------------+-------------+------------------+------------------+
|  1960|     4134930|     45160900|     106180|            236720|     1633510|                2158520|     13321100|          26547700|           5292100|
|  2010|     6072017|     44095950|      72867|            421059|     1749809|                3764142|     10125170|          30401698|           3569080|
|  1970|     9607930|     91383800|     192230|            554570|     4159020|                4702120|     28486000|          53157800|           9739900|
|  1980|    14074328|    117048900|     206439|            86563

### Step 9. What is the most dangerous decade to live in the US?

In [53]:
decade_df.withColumn("Total", sum(map(lambda c: f.col(f"sum({c})"), data_cols))).orderBy(f.desc("Total")).limit(1).show()

+------+------------+-------------+-----------+------------------+------------+-----------------------+-------------+------------------+------------------+---------+
|decade|sum(Violent)|sum(Property)|sum(Murder)|sum(Forcible_Rape)|sum(Robbery)|sum(Aggravated_assault)|sum(Burglary)|sum(Larceny_Theft)|sum(Vehicle_Theft)|    Total|
+------+------------+-------------+-----------+------------------+------------+-----------------------+-------------+------------------+------------------+---------+
|  1990|    17527048|    119053499|     211664|            998827|     5748930|               10568963|     26750015|          77679366|          14624418|273162730|
+------+------------+-------------+-----------+------------------+------------+-----------------------+-------------+------------------+------------------+---------+

